In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import datetime as datetime
from datetime import datetime as dt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.chrome.options import Options

dictCalander = {"January":"01" ,
"February":"02", 
"March":"03", 
"April":"04", 
"May":"05", 
"June":"06", 
"July":"07", 
"August":"08", 
"September":"09", 
"October":"10",
"November":"11",
"December":"12"}

In [4]:
def dataLoad():
    return pd.read_excel("./Location.xlsx").applymap(str.lower)

In [5]:
def cheapflightsearch(startlocation ,endlocation , startdate , enddate,dataframe):
    airline = "";
    price = "";
    if(not((isValidLocation(dataframe,startlocation)) & (isValidLocation(dataframe,endlocation)))):
        return ("Invalid Location" , "-1")
    elif(not ((isValidDateFormat(startdate)) & ((isValidDateFormat(enddate) | (enddate is None) | (enddate ==""))))):
        return ("Invalid Date" , "-1")
    return (airline , price)

In [6]:
def isValidLocation(dataframe , location):
    return (location.lower() in dataframe['Name'].unique()) | (location.lower() in dataframe['Code'].unique())

In [7]:
def isValidDateFormat(dateValue):
    format = "%Y-%m-%d"
    present = dt.now().strftime(format)
    present = datetime.datetime.strptime(present, format)
    if dateValue is None:
        return False
    else:
        try:
            past = datetime.datetime.strptime(dateValue, format)
            if past>present:
                return True
            else:
                return False
        except ValueError:
            return False

In [17]:
def callCheapFlightStatus(source,destination,startDate,endDate):
    dataframe = dataLoad()
    result = cheapflightsearch(source,destination,startDate,endDate,dataframe)
    if (result[0]==""):
        chrome_options = Options()
        chrome_options.add_experimental_option("detach", True)
        driver = webdriver.Chrome('./chromedriver', options=chrome_options)
        driver.get("https://www.makemytrip.com/")
        try:
            driver.maximize_window()
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html')))
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/label/input").click()
            time.sleep(3)
            sourceFiled= driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/input")
            sourceFiled.clear()
            sourceFiled.send_keys(source)
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/div/div[1]/ul/li[1]/div/div[1]").click()
            time.sleep(3)
            destinationPath=driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input")
            destinationPath.clear()
            destinationPath.send_keys(destination)
            time.sleep(3)
            driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/div/div[1]/ul/li[1]/div/div[1]").click()
            time.sleep(3)
            currentmonth = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[2]/div[1]/div[1]/div").text
            print("Inside the Driver " )
            pastmonth = dictCalander.get(currentmonth.split()[0])
            startMonth=startDate.split("-")[1]
            startDay=startDate.split("-")[2]
            print("Past Month " +pastmonth)
            print("Current Month "+startMonth)
            while(pastmonth!=currentmonth):
                driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[3]/div[1]/div/div/div/div[2]/div/div[1]/span[2]").click()
        except e:
            print("Error - " +e)
        finally:
            #driver.close()
            print("Executed")
    else:
        print(result)

In [18]:
callCheapFlightStatus("pnq","ccu","2022-07-02","")

C:\Users\WWWABC~1.IN\AppData\Local\Temp/ipykernel_21660/681998398.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=chrome_options)
C:\Users\WWWABC~1.IN\AppData\Local\Temp/ipykernel_21660/681998398.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/label/input").click()
C:\Users\WWWABC~1.IN\AppData\Local\Temp/ipykernel_21660/681998398.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  sourceFiled= driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/div[1]/div/div/div/input")
C:\Users\WWWABC~1.IN\AppData\Local\Temp/ipykernel_21660/681998398.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_eleme

Inside the Driver 
Past Month 06
Current Month 07
Executed


NameError: name 'e' is not defined